# Fintech final project baseline 
-----
本 notebook 作為 fintech 金融科技導論的期末專題競賽 baseline 程式說明。

* [競賽連結](https://tbrain.trendmicro.com.tw/Competitions/Details/24)

首先會就資料格式以及處理說明，接續簡介模型訓練,最終預測結果並輸出目標格式。

Reminder: XGBoost 版本會影響 performance，請同學多注意。

In [2]:
!pip install xgboost==1.7.1

In [3]:
# import library
import os
import pandas as pd
import numpy as np
import time
import collections
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import TomekLinks

## 資料前處理
這邊針對訓練資料和測試的資料作整理。
baseline主要會使用到的csv檔案如下:
  - public_train_custinfo_full_hashed.csv: 包含主要要判斷的alert key對應的幾項參數，顧客id, 風險等級, 職業, 行內總資產和年齡。
  - train_x_alert_date: 作為訓練資料的alert key以及發生日期，共23906筆。
  - public_x_alert_date: 作為公開測試集的alert key，格式同上共1845筆。
  - train_y_answer: 訓練資料alert key對應最後是否SAR。
  - 預測的案件名單及提交檔案範例: 用於生成預測結果

除此之外，還會使用到顧客資訊當作訓練資料:
  - public_train_x_ccba_full_hashed.csv
  - public_train_x_cdtx0001_full_hashed.csv
  - public_train_x_dp_full_hashed.csv
  - public_train_x_remit1_full_hashed.csv

前處理的方式包含:
  - 從 alert key 檢索出顧客資訊
  - 對非數值 feature 做 label encoding
  - 從顧客資訊中挑選適合的 features 當作訓練資料，這裡挑選離 alert date 最近的一筆顧客資訊當作 features
  - 統計 training data 缺失值數量

In [4]:
def preprocess(data_dir):
    # declare csv path
    train_alert_date_csv = os.path.join(data_dir, 'train_x_alert_date.csv')
    cus_info_csv = os.path.join(data_dir, 'public_train_x_custinfo_full_hashed.csv')
    y_csv = os.path.join(data_dir, 'train_y_answer.csv')

    ccba_csv = os.path.join(data_dir, 'public_train_x_ccba_full_hashed.csv')
    cdtx_csv = os.path.join(data_dir, 'public_train_x_cdtx0001_full_hashed.csv')
    dp_csv = os.path.join(data_dir, 'public_train_x_dp_full_hashed.csv')
    remit_csv = os.path.join(data_dir, 'public_train_x_remit1_full_hashed.csv')

    public_x_csv = os.path.join(data_dir, 'public_x_alert_date.csv')

    # private
    private_cus_info_csv = os.path.join(data_dir, 'private_x_custinfo_full_hashed.csv')
    private_ccba_csv = os.path.join(data_dir, 'private_x_ccba_full_hashed.csv')
    private_cdtx_csv = os.path.join(data_dir, 'private_x_cdtx0001_full_hashed.csv')
    private_dp_csv = os.path.join(data_dir, 'private_x_dp_full_hashed.csv')
    private_remit_csv = os.path.join(data_dir, 'private_x_remit1_full_hashed.csv')

    private_x_csv = os.path.join(data_dir, 'private_x_alert_date.csv')

    cus_csv = [ccba_csv, cdtx_csv, dp_csv, remit_csv]
    private_cus_csv = [private_ccba_csv, private_cdtx_csv, private_dp_csv, private_remit_csv]

    date_col = ['byymm', 'date', 'tx_date', 'trans_date']
    data_use_col = [[1,3,4,5,6,7,8,9],[2,3,4],[1,4,5,6,7,8,9,10,11],[2,3]]
    
    print('Reading csv...')
    # read csv
    df_y = pd.read_csv(y_csv)
    df_cus_info = pd.concat([pd.read_csv(cus_info_csv), pd.read_csv(private_cus_info_csv)],axis=0, ignore_index=True)
    df_date = pd.read_csv(train_alert_date_csv)
    cus_data = [pd.concat([pd.read_csv(_x), pd.read_csv(_y)],axis=0, ignore_index=True) for _x, _y in zip(cus_csv, private_cus_csv)]
    df_public_x = pd.concat([pd.read_csv(public_x_csv), pd.read_csv(private_x_csv)],axis=0, ignore_index=True)

    # do label encoding
    le = LabelEncoder()
    cus_data[2].debit_credit = le.fit_transform(cus_data[2].debit_credit)

    cnts = [0] * 4
    labels = []
    training_data = []

    print('Start processing training data...')
    start = time.time()
    for i in range(df_y.shape[0]):
        # from alert key to get customer information
        cur_data = df_y.iloc[i]
        alert_key, label = cur_data['alert_key'], cur_data['sar_flag']

        cus_info = df_cus_info[df_cus_info['alert_key']==alert_key].iloc[0]
        cus_id = cus_info['cust_id']
        cus_features = cus_info.values[2:]

        date = df_date[df_date['alert_key']==alert_key].iloc[0]['date']


        cnt = 0
        for item, df in enumerate(cus_data):
            cus_additional_info = df[df['cust_id']==cus_id]
            cus_additional_info = cus_additional_info[cus_additional_info[date_col[item]]<=date]

            if cus_additional_info.empty:
                cnts[item] += 1
                len_item = len(data_use_col[item])
                if item == 2:
                    len_item -= 1
                cus_features = np.concatenate((cus_features, [np.nan] * len_item), axis=0)
            else:
                cur_cus_feature = cus_additional_info.loc[cus_additional_info[date_col[item]].idxmax()]
                
                cur_cus_feature = cur_cus_feature.values[data_use_col[item]]
                # 處理 實際金額 = 匯率*金額
                if item == 2:
                    cur_cus_feature = np.concatenate((cur_cus_feature[:2], [cur_cus_feature[2]*cur_cus_feature[3]], cur_cus_feature[4:]), axis=0)
                cus_features = np.concatenate((cus_features, cur_cus_feature), axis=0)
        labels.append(label)
        training_data.append(cus_features)
        print('\r processing data {}/{}'.format(i+1, df_y.shape[0]), end = '')
    print('Processing time: {:.3f} secs'.format(time.time()-start))
    print('Missing value of 4 csvs:', cnts)


    print('Start processing testing data')
    testing_data, testing_alert_key = [], []
    for i in range(df_public_x.shape[0]):
        # from alert key to get customer information
        cur_data = df_public_x.iloc[i]
        alert_key, date = cur_data['alert_key'], cur_data['date']

        cus_info = df_cus_info[df_cus_info['alert_key']==alert_key].iloc[0]
        cus_id = cus_info['cust_id']
        cus_features = cus_info.values[2:]

        for item, df in enumerate(cus_data):
            cus_additional_info = df[df['cust_id']==cus_id]
            cus_additional_info = cus_additional_info[cus_additional_info[date_col[item]]<=date]

            if cus_additional_info.empty:
                len_item = len(data_use_col[item])
                if item == 2:
                    len_item -= 1
                cus_features = np.concatenate((cus_features, [np.nan] * len_item), axis=0)
            else:
                cur_cus_feature = cus_additional_info.loc[cus_additional_info[date_col[item]].idxmax()]
                cur_cus_feature = cur_cus_feature.values[data_use_col[item]]
                # 處理 實際金額 = 匯率*金額
                if item == 2:
                    cur_cus_feature = np.concatenate((cur_cus_feature[:2], [cur_cus_feature[2]*cur_cus_feature[3]], cur_cus_feature[4:]), axis=0)
                cus_features = np.concatenate((cus_features, cur_cus_feature), axis=0)

        testing_data.append(cus_features)
        testing_alert_key.append(alert_key)
        # print(cus_features)
        print('\r processing data {}/{}'.format(i+1, df_public_x.shape[0]), end = '')
    return np.array(training_data), labels, np.array(testing_data), testing_alert_key

# 訓練資料處理

In [5]:
data_dir = './data'
# data preprocessing
training_data, labels, testing_data, testing_alert_key = preprocess(data_dir)

Reading csv...
Start processing training data...
 processing data 121/23906

KeyboardInterrupt: 

## 缺失值補漏
  可以發現有不少筆資料其實是有缺漏的，補上缺失值的方法有很多種，我們對於數值類資料補上中位數，對於類別類資料補上眾數。

In [ ]:
''' Missing Value Imputation '''
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
imp_most_frequent = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
# for numerical index we do imputation using median
numerical_index = [2,4,5,6,7,8,9,10,11,14,17,24]
# Otherwise we select the most frequent
non_numerical_index = [0,1,3,12,13,15,16,18,19,20,21,22,23]

numerical_data = training_data[:, numerical_index]
non_numerical_data = training_data[:, non_numerical_index]

imp_median.fit(numerical_data)
numerical_data = imp_median.transform(numerical_data)

imp_most_frequent.fit(non_numerical_data)
non_numerical_data = imp_most_frequent.transform(non_numerical_data)

training_data = np.concatenate((non_numerical_data, numerical_data), axis=1)

  此外，若類別類資料跟數字大小沒關係，我們採用 one-hot encoding 將其編碼。

In [ ]:
# for some catogorical features, we do one hot encoding
one_hot_index = [1,3,4,5,6,7,8,9,12]
onehotencorder = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'), one_hot_index)],
    remainder='passthrough'                     
)
onehotencorder.fit(training_data)
training_data = onehotencorder.transform(training_data)
print(training_data.shape)

(23906, 357)


## 下採樣

In [ ]:
# Undersampling: Tomek Links
tl = TomekLinks()
training_data, labels = tl.fit_resample(training_data, labels)
print(training_data[0])
print(training_data.shape, testing_data.shape)
tl = TomekLinks()
training_data, labels = tl.fit_resample(training_data, labels)
print(training_data[0])
print(training_data.shape, testing_data.shape)

  (0, 17)	1.0
  (0, 72)	1.0
  (0, 93)	1.0
  (0, 97)	1.0
  (0, 99)	1.0
  (0, 102)	1.0
  (0, 125)	1.0
  (0, 141)	1.0
  (0, 336)	1.0
  (0, 341)	1.0
  (0, 342)	4.0
  (0, 344)	1.0
  (0, 345)	375576.0
  (0, 346)	85428.0
  (0, 347)	301224.0
  (0, 348)	154122.0
  (0, 352)	151434.0
  (0, 354)	673.0
  (0, 355)	309.0
  (0, 356)	37421.0
(23821, 357) (3850, 25)
  (0, 17)	1.0
  (0, 72)	1.0
  (0, 93)	1.0
  (0, 97)	1.0
  (0, 99)	1.0
  (0, 102)	1.0
  (0, 125)	1.0
  (0, 141)	1.0
  (0, 336)	1.0
  (0, 341)	1.0
  (0, 342)	4.0
  (0, 344)	1.0
  (0, 345)	375576.0
  (0, 346)	85428.0
  (0, 347)	301224.0
  (0, 348)	154122.0
  (0, 352)	151434.0
  (0, 354)	673.0
  (0, 355)	309.0
  (0, 356)	37421.0
(23793, 357) (3850, 25)


# XGBoost 訓練

In [ ]:

import xgboost as xgb
# 建立 XGBClassifier 模型
xgbrModel=xgb.XGBClassifier(random_state=0)
# 使用訓練資料訓練模型
# xgbrModel.fit(training_data, labels)


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
learning_rate=[round(float(x),2) for x in np.linspace(start=0.01, stop=0.2, num=10)]
colsample_bytree =[round(float(x),2) for x in np.linspace(start=0.1, stop=1, num=10)]

random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'learning_rate': learning_rate,
               'colsample_bytree': colsample_bytree}
xgbrModel = RandomizedSearchCV(estimator = xgbrModel, param_distributions=random_grid,
                              n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

xgbrModel.fit(training_data, labels)
print(xgbrModel.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END colsample_bytree=0.1, learning_rate=0.16, max_depth=30, n_estimators=200; total time=   1.4s
[CV] END colsample_bytree=0.1, learning_rate=0.16, max_depth=30, n_estimators=200; total time=   1.5s
[CV] END colsample_bytree=0.1, learning_rate=0.16, max_depth=30, n_estimators=200; total time=   1.6s
[CV] END colsample_bytree=0.5, learning_rate=0.09, max_depth=None, n_estimators=200; total time=   2.7s
[CV] END colsample_bytree=0.5, learning_rate=0.09, max_depth=None, n_estimators=200; total time=   2.7s
[CV] END colsample_bytree=0.5, learning_rate=0.09, max_depth=None, n_estimators=200; total time=   2.7s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=80, n_estimators=200; total time=   5.7s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=80, n_estimators=200; total time=   6.3s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=80, n_estimators=200; total time=   6.5s
[CV] END cols

# 預測與結果輸出
  利用訓練好的模型對目標alert key預測報SAR的機率以及輸出為目標格式。
  目標輸出筆數3850，其中public筆數為1845筆。
  因上傳格式需要private跟public alert key皆考慮，直接從預測範本統計要預測的alert key，預測結果輸出為prediction.csv。

In [ ]:
# Do missing value imputation and one-hot encoding for testing data
test_numerical_data = testing_data[:, numerical_index]
test_non_numerical_data = testing_data[:, non_numerical_index]

test_numerical_data = imp_median.transform(test_numerical_data)

test_non_numerical_data = imp_most_frequent.transform(test_non_numerical_data)

testing_data = np.concatenate((test_non_numerical_data, test_numerical_data), axis=1)
testing_data = onehotencorder.transform(testing_data)

In [ ]:
# Read csv of all alert keys need to be predicted
public_private_test_csv = os.path.join(data_dir, '預測的案件名單及提交檔案範例.csv')
df_public_private_test = pd.read_csv(public_private_test_csv)

# Predict probability
predicted = []
for i, _x in enumerate(xgbrModel.predict_proba(testing_data)):
    predicted.append([testing_alert_key[i], _x[1]])
predicted = sorted(predicted, key= lambda s: s[1])

# 考慮private alert key部分，滿足上傳條件
public_private_alert_key = df_public_private_test['alert_key'].values
print(len(public_private_alert_key))

# For alert key not in public, add zeros
for key in public_private_alert_key:
    if key not in testing_alert_key:
        predicted.append([key, 0])

predict_alert_key, predict_probability = [], []
for key, prob in predicted:
    predict_alert_key.append(key)
    predict_probability.append(prob)

df_predicted = pd.DataFrame({
    "alert_key": predict_alert_key,
    "probability": predict_probability
})

df_predicted.to_csv('result.csv', index=False)

3850


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def score(model, x, y):
    pred = model.predict(x)
    print('Result:')
    print(f"Accuracy Score: {accuracy_score(y, pred)*100:}%")
    print(f"Precision Score: {precision_score(y, pred)*100:.2f}%")
    print(f"Recall Score: {recall_score(y, pred)*100:.2f}%")
    print(f"F1 score: {f1_score(y, pred)*100:.2f}%")
    print(f"Confusion Matrix:\n {confusion_matrix(y, pred)}")

def read_public_answer(data_dir):
    y_csv = os.path.join(data_dir, '24_ESun_public_y_answer.csv')
    df_y = pd.read_csv(y_csv)
    return df_y.iloc[:, 1]

public_label = read_public_answer('./data')
score(xgbrModel, testing_data[:1845], public_label)

Result:
Accuracy Score: 99.40379403794037%
Precision Score: 0.00%
Recall Score: 0.00%
F1 score: 0.00%
Confusion Matrix:
 [[1834    0]
 [  11    0]]


/home/yslin/anaconda3/envs/lab/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# SAR

In [6]:
def getResultFromAlertKey(alert_key, df_truth, found, catch):
  for i , answer_key in enumerate(df_truth.iloc[:, 0]):
    answer_val = df_truth.iloc[i][1]
    if alert_key == int(answer_key) and answer_val == 1:
      print(alert_key, found, catch)
      return 1
  return 0

# pos = []
# def getInfoFromAlertKey(alert_key, ):
#   df = df_training_with_key
#   for i , answer_key in enumerate(df.iloc[:, 0]):
#     if alert_key == int(answer_key):
#       # print(df.iloc[i, :])
#       pos.append(df.iloc[i, :].values)
#       break
#   return

def sar():
  df_truth = pd.read_csv('data/24_ESun_public_y_answer.csv')
  df_prediction = pd.read_csv('prediction.csv')
  
  print(df_truth.shape)
  print(df_prediction)

  N = 0
  for i , val in enumerate(df_truth.iloc[:, 1]):
    if val == 1:
      N += 1 
  print(N)

  found = 0
  catch = 0
  for i , item in enumerate(df_prediction['alert_key']):
    # getInfoFromAlertKey(int(item), df_truth)
    found += getResultFromAlertKey(int(item), df_truth, found, catch)
    catch += 1
    if found == N-1:
      break
  print('found = ', found)
  print('catch = ', catch)
  print('SAR Score = ', found/catch)

sar()
# df_pos = pd.DataFrame(pos, columns = df_training_with_key.columns.tolist())
# print(df_pos)

(1845, 2)
      alert_key  probability
0        353220     0.070268
1        360572     0.070268
2        361565     0.070268
3        367723     0.070268
4        372107     0.070268
...         ...          ...
3845     375610     0.261708
3846     372351     0.262284
3847     373827     0.263037
3848     378054     0.267724
3849     376670     0.335035

[3850 rows x 2 columns]
11
355152 0 1688
361617 1 1812
359668 2 2088
358453 3 2098
356602 4 2388
355724 5 2418
363033 6 2745
355091 7 2895
354939 8 3148
363320 9 3248
found =  10
catch =  3249
SAR Score =  0.0030778701138811943
